# Simulation plugins


![](https://i.imgur.com/RSOTDIN.png)

Sparameters are common in RF and photonic simulation.



```bash

         top view
              ________________________________
             |                               |
             | xmargin_left                  | port_extension
             |<--------->       port_margin ||<-->
          o2_|___________          _________||_o3
             |           \        /          |
             |            \      /           |
             |             ======            |
             |            /      \           |
          o1_|___________/        \__________|_o4
             |   |                 <-------->|
             |   |ymargin_bot   xmargin_right|
             |   |                           |
             |___|___________________________|

        side view
              ________________________________
             |                     |         |
             |                     |         |
             |                   zmargin_top |
             |xmargin_left         |         |
             |<---> _____         _|___      |
             |     |     |       |     |     |
             |     |     |       |     |     |
             |     |_____|       |_____|     |
             |       |                       |
             |       |                       |
             |       |zmargin_bot            |
             |       |                       |
             |_______|_______________________|



```

We are going to simulate a MZI interferometer circuit. For that we need to simulate each of the component Sparameters in Meep and then use a linear circuit solver to solve the Sparameters for the circuit.

## Mode solver


In [ ]:
from gdsfactory.simulation.modes import find_modes_waveguide

def silicon_index(wl):
    """ a rudimentary silicon refractive index model """
    a = 0.2411478522088102
    b = 3.3229394315868976
    return a / wl + b

nm = 1e-3
wl = 1.55
w = 500*nm
modes = find_modes_waveguide(wavelength=wl, wg_width=w, mode_number=1, wg_thickness=0.22, slab_thickness=0.0, ncore=silicon_index(wl), nclad=1.4)

In [ ]:
mode = modes[1]

In [ ]:
mode.plot_e_all()

In [ ]:
mode.neff

## FDTD

In [ ]:
import gdsfactory.simulation.gmeep as gm
import gdsfactory.simulation as sim
import gdsfactory as gf

import ubcpdk as pdk

In [ ]:
c = pdk.components.ebeam_y_1550()
c.unlock()
c.auto_rename_ports()
c

In [ ]:
c.ports

`run=False` only plots the simulations for you to review that is set up **correctly**

In [ ]:
df = gm.write_sparameters_meep(c, run=False)

In [ ]:
df = gm.write_sparameters_meep(c, run=True)

In [ ]:
sim.plot.plot_sparameters(df, keys=['s21m'])

## 2.5D FDTD

For faster simulations you can do an effective mode approximation, to compute the mode of the slab and run a 2D simulation to speed your [simulations](https://www.lumerical.com/learn/whitepapers/lumericals-2-5d-fdtd-propagation-method/)

In [ ]:
ncore = sim.get_effective_indices(
            ncore=3.4777,
            ncladding=1.444,
            nsubstrate=1.444,
            thickness=0.22,
            wavelength=1.55,
            polarization="te",
        )[0]
ncore

In [ ]:
df2d = gm.write_sparameters_meep(c, resolution=20, is_3d=False, material_name_to_meep=dict(si=ncore))

In [ ]:
gf.simulation.plot.plot_sparameters(df2d)

In [ ]:
sim.plot.plot_sparameters(df2d, keys=['s21m'])
sim.plot.plot_sparameters(df, keys=['s21m'])

For a small taper S21 (Transmission) is around 0dB (100% transmission)

## Port symmetries

You can save some simulations in reciprocal devices.
If the device looks the same going from in -> out as out -> in, we only need to run one simulation

In [ ]:
c = gf.components.bend_euler(radius=3)
c

In [ ]:
df = gm.write_sparameters_meep_1x1_bend90(c, run=False)

In [ ]:
df = gm.write_sparameters_meep_1x1_bend90(c, run=True)

In [ ]:
gf.simulation.plot.plot_sparameters(df)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=("s21m",), logscale=False)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=("s11m",))

In [ ]:
c = pdk.components.ebeam_crossing4(decorator=gf.port.auto_rename_ports)
c

Here are the port symmetries for a crossing

```python
port_symmetries = {
    "o1": {
        "s11": ["s22", "s33", "s44"],
        "s21": ["s12", "s34", "s43"],
        "s31": ["s13", "s24", "s42"],
        "s41": ["s14", "s23", "s32"],
    }
}
```

In [ ]:
df = gm.write_sparameters_meep(
    c,
    resolution=20,
    ymargin=0,
    port_symmetries=gm.port_symmetries.port_symmetries_crossing,
    run=False,
)

In [ ]:
df = gm.write_sparameters_meep(
    c,
    resolution=20,
    ymargin=0,
    port_symmetries=gm.port_symmetries.port_symmetries_crossing,
    run=True,
)

In [ ]:
gm.plot.plot_sparameters(df)

In [ ]:
gm.plot.plot_sparameters(df, keys=("s31m",))

## Multicore (MPI)

You can divide each simulation into multiple cores thanks to [MPI (message passing interface)](https://en.wikipedia.org/wiki/Message_Passing_Interface)

Lets try to reproduce the coupler results from the [Meep docs](https://meep.readthedocs.io/en/latest/Python_Tutorials/GDSII_Import/)

According to the simulations in the doc to get a 3dB (50%/50%) splitter you need 150nm over 8um

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

import gdsfactory as gf
import gdsfactory.simulation as sim
import gdsfactory.simulation.gmeep as gm

In [ ]:
c = gf.components.coupler(length=8, gap=0.13)
c

In [ ]:
gm.write_sparameters_meep(component=c, run=False)

In [ ]:
filepath = gm.write_sparameters_meep_mpi(
    component=c,
    cores=4,
    resolution=30,
)

In [ ]:
df = pd.read_csv(filepath)

In [ ]:
gf.simulation.plot.plot_sparameters(df)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=["s13m", "s14m"])

## Batch

You can also run a batch of multicore simulations

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gdsfactory as gf

import gdsfactory.simulation as sim
import gdsfactory.simulation.gmeep as gm

In [ ]:
c = gf.components.straight(length=3.1)

In [ ]:
gm.write_sparameters_meep(c, ymargin=3, run=False)

In [ ]:
c1_dict = {"component": c, "ymargin": 3}
jobs = [
    c1_dict,
]

filepaths = gm.write_sparameters_meep_batch_1x1(
    jobs=jobs,
    cores_per_run=4,
    total_cores=8,
    lazy_parallelism=True,
)

In [ ]:
df = pd.read_csv(filepaths[0])
gf.simulation.plot.plot_sparameters(df)

In [ ]:
c = gf.components.coupler_ring()
c

In [ ]:
p = 2.5
gm.write_sparameters_meep(c, ymargin=0, ymargin_bot=p, xmargin=p, run=False)

In [ ]:
c1_dict = dict(
    component=c,
    ymargin=0,
    ymargin_bot=p,
    xmargin=p,
)
jobs = [c1_dict]

filepaths = gm.write_sparameters_meep_batch(
    jobs=jobs,
    cores_per_run=4,
    total_cores=8,
    delete_temp_files=False,
    lazy_parallelism=True,
)

In [ ]:
df = pd.read_csv(filepaths[0])

In [ ]:
gm.plot.plot_sparameters(df)

In [ ]:
gm.plot.plot_sparameters(df, keys=["s31m", "s41m"])

In [ ]:
gm.plot.plot_sparameters(df, keys=["s31m"])

In [ ]:
gm.plot.plot_sparameters(df, keys=["s41m"])

## 3D rendering

In [ ]:
from gdsfactory.simulation.add_simulation_markers import add_simulation_markers
import ubcpdk as pdk

y = pdk.components.ebeam_y_1550()
y.unlock()
y.auto_rename_ports()
y = add_simulation_markers(y)
y

In [ ]:
scene = y.to_3d()
scene.show()

## Circuit simulation


### 1. Circuit simulator: simphony

In [ ]:
import gdsfactory.simulation.simphony as gs
import gdsfactory.simulation.gmeep as gm
import gdsfactory as gf
import ubcpdk as pdk

In [ ]:
y = pdk.components.ebeam_y_1550()
y.unlock()
y.auto_rename_ports()
y

In [ ]:
df = gm.write_sparameters_meep(y)

In [ ]:
mzi10 = gf.components.mzi(splitter=y, delta_length=10)
mzi10

In [ ]:
from gdsfactory.simulation.simphony.components import model_factory

ebeam_y_1550 = gf.partial(gs.model_from_csv, filepath=df, pins=y.ports.keys())
model_factory.update(ebeam_y_1550=ebeam_y_1550)

In [ ]:
mzi10_circuit = gs.component_to_circuit(mzi10, model_factory=model_factory)

In [ ]:
gs.plot_circuit(
    mzi10_circuit,
    start=1500e-9,
    stop=1600e-9,
    logscale=True,
)

In [ ]:
mzi20 = gf.components.mzi(splitter=y, delta_length=20)
mzi20

In [ ]:
mzi20_circuit = gs.component_to_circuit(mzi20, model_factory=model_factory)
gs.plot_circuit(
    mzi20_circuit,
    start=1500e-9,
    stop=1600e-9,
    logscale=True,
)

### Montecarlo variability simulation

As you define your circuit models you can also include the standard deviation for each component model.

For example for the waveguide model you can define:

```
sigma_ne: Standard deviation of the effective index for monte carlo simulations (default 0.05).
sigma_ng: Standard deviation of the group velocity for monte carlo simulations
sigma_nd: Standard deviation of the group dispersion for monte carlo simulations
```

And you can pass `runs` to change the number of Monte Carlo iterations to run.

The variability values are foundry specific and they relate to the width and sigma variations from the fabrication process.

In [ ]:
gs.plot_circuit_montecarlo(mzi10_circuit, runs=10)

### 2. Circuit simulator: SAX

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import jax.numpy as jnp
from omegaconf import OmegaConf
import sax
from pprint import pprint

import gdsfactory as gf
import gdsfactory.simulation.sax as gsax
import gdsfactory.simulation.gmeep as gm

import ubcpdk as pdk

In [ ]:
y = pdk.components.ebeam_y_1550()
y.unlock()
y.auto_rename_ports()
y

In [ ]:
mzi = gf.components.mzi(splitter=y, delta_length=10)
mzi

In [ ]:
def straight(wl=1.5, length=10.0, neff=2.4) -> sax.SDict:
    wl0 = 1.5  # center wavelength for which the waveguide model is defined
    return sax.reciprocal({("o1", "o2"): jnp.exp(2j * jnp.pi * neff * length / wl)})


def bend_euler(wl=1.5, length=20.0):
    """ "Let's assume a reduced transmission for the euler bend compared to a straight"""
    return {k: 0.99 * v for k, v in straight(wl=wl, length=length).items()}


In [ ]:
df = gm.write_sparameters_meep(y, run=True)
#ebeam_y_1550 = gsax.read.sdict_from_csv(filepath=df)
ebeam_y_1550 = gsax.read.model_from_csv(filepath=df)

In [ ]:
netlist = mzi.get_netlist_dict()
circuit = sax.circuit_from_netlist(
    netlist=netlist,
    models={
        "bend_euler": bend_euler,
        "ebeam_y_1550": ebeam_y_1550,
        "straight": straight,
    },
)

In [ ]:
wl = np.linspace(1.5, 1.6)
S = circuit(wl=wl)
plt.figure(figsize=(14, 4))
plt.title("MZI")
plt.plot(1e3 * wl, 10*np.log10(jnp.abs(S["o1", "o2"]) ** 2))
plt.xlabel("λ [nm]")
plt.ylabel("T")
plt.grid(True)
plt.show()

In [ ]:
mzi = gf.components.mzi(splitter=y, delta_length=20)
mzi

In [ ]:
netlist = mzi.get_netlist_dict()
circuit = sax.circuit_from_netlist(
    netlist=netlist,
    models={
        "bend_euler": bend_euler,
        "ebeam_y_1550": ebeam_y_1550,
        "straight": straight,
    },
)

In [ ]:
wl = np.linspace(1.5, 1.6)
S = circuit(wl=wl)
plt.figure(figsize=(14, 4))
plt.title("MZI")
plt.plot(1e3 * wl, 10*np.log10(jnp.abs(S["o1", "o2"]) ** 2))
plt.xlabel("λ [nm]")
plt.ylabel("T")
plt.grid(True)
plt.show()